In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!pip install scanpy --quiet

     |████████████████████████████████| 2.0 MB 5.3 MB/s 
     |████████████████████████████████| 9.4 MB 65.6 MB/s 
     |████████████████████████████████| 96 kB 5.0 MB/s 
     |████████████████████████████████| 88 kB 7.1 MB/s 
     |████████████████████████████████| 965 kB 70.0 MB/s 
     |████████████████████████████████| 295 kB 75.9 MB/s 
     |████████████████████████████████| 1.1 MB 61.1 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 


In [3]:
import random
import sys
import os
import anndata as ad
from argparse import Namespace

config = Namespace(
    # LEARNING_RATE = 0.00002,
    DEVICE = 'cpu',
    BATCH_SIZE = 100,
    NUM_WORKERS = 4,
    N_GENES = 13431,
    N_PEAKS = 116465,
    MAX_SEQ_LEN_GEX = 1500,
    MAX_SEQ_LEN_ATAC = 15000,
)

In [4]:
execfile("drive/MyDrive/Colab_Notebooks/CPSC532S/final_project/code/resources/data.py")
execfile("drive/MyDrive/Colab_Notebooks/CPSC532S/final_project/code/resources/models.py")
execfile("drive/MyDrive/Colab_Notebooks/CPSC532S/final_project/code/resources/utils.py")

## Load data

In [5]:
index = get_chr_index(ad.read_h5ad("drive/MyDrive/Colab_Notebooks/CPSC532S/final_project/data/ATAC_processed.h5ad"))

In [6]:
gc.collect()

242

In [7]:
batch = ad.read_h5ad("drive/MyDrive/Colab_Notebooks/CPSC532S/final_project/data/GEX_processed.h5ad").obs['batch']
batch = list(batch)
# train_id = [a for a, l in enumerate(batch) if l not in ['s2d4','s1d1']]
# val_id =  [a for a, l in enumerate(batch) if l == 's1d1']
test_id = [a for a, l in enumerate(batch) if l == 's2d4']

In [8]:
del batch
gc.collect()

280

In [9]:
cell_type_all = ad.read_h5ad("drive/MyDrive/Colab_Notebooks/CPSC532S/final_project/data/GEX_processed.h5ad").obs['cell_type']
gc.collect()
csr_gex = ad.read_h5ad("drive/MyDrive/Colab_Notebooks/CPSC532S/final_project/data/GEX_processed.h5ad").layers['log_norm']
gc.collect()
csr_atac = ad.read_h5ad("drive/MyDrive/Colab_Notebooks/CPSC532S/final_project/data/ATAC_processed.h5ad").layers['log_norm']
gc.collect()

242

In [10]:
random.seed(0)

idx_test = test_id  # Full dataset
gex_test = csr_gex[idx_test,:]
atac_test = csr_atac[idx_test,:]
cell_type_test = [cell_type_all[j] for j in idx_test]

data_test = get_dataloaders(gex_test, atac_test, cell_type_test)

## Parameter set 6

In [11]:
config.ALPHA = 0.8
config.MARGIN = 0.5
config.N_CHANNELS = 64

Null model:

In [12]:
model = Encoder(kernel_size_gex = 100, kernel_size_atac_1 = 30, kernel_size_atac_2 = 5, index = index).to(config.DEVICE) ## CHANGED TO SMALLER KERNAL SIZE FOR ATAC
criterion = bidirectTripletLoss(alpha = config.ALPHA, margin = config.MARGIN).to('cpu')
loss_cross, loss_triplet, ct_match_prob, cell_match_prob = inference(model, criterion, data_test)
print('cross entropy loss = {0}, triplet loss = {1}, \n cell type match prob = {2}, cell match prob = {3}'.format(
    round(loss_cross, 3),
    round(loss_triplet, 3), 
    round(ct_match_prob, 3),
    round(cell_match_prob, 3)
    )
)

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


cross entropy loss = 4.572, triplet loss = 0.909, 
 cell type match prob = 0.061, cell match prob = 0.011


Trained model:

In [13]:
file = 'drive/MyDrive/Colab_Notebooks/CPSC532S/final_project/model/trained_model_semihard_allcells_alpha' + \
        str(config.ALPHA) + '_margin' + str(config.MARGIN) + '_nchannels' + str(config.N_CHANNELS) + '_15epochs'
model.load_state_dict(torch.load(file, map_location=torch.device('cpu')))
loss_cross, loss_triplet, ct_match_prob, cell_match_prob = inference(model, criterion, data_test)
print('cross entropy loss = {0}, triplet loss = {1}, \n cell type match prob = {2}, cell match prob = {3}'.format(
    round(loss_cross, 3),
    round(loss_triplet, 3), 
    round(ct_match_prob, 3),
    round(cell_match_prob, 3)
    )
)

cross entropy loss = 3.672, triplet loss = 0.525, 
 cell type match prob = 0.406, cell match prob = 0.029
